COVID-19 Data Cleaning

Inspirated by https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6

In [11]:
import pandas as pd
import numpy as np

In [12]:
confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('time_series_covid19_recovered_global.csv')

In [13]:
dates = confirmed_df.columns[4:]
confirmed_df_long = confirmed_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_vars=dates, var_name='Date', value_name='Confirmed') 
deaths_df_long = deaths_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_vars=dates, var_name='Date', value_name='Deaths')
recovered_df_long = recovered_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_vars=dates, var_name='Date', value_name='Recovered')

In [14]:
deaths_df_long.head()

,Province/State,Country/Region,Lat,Long,Date,Deaths
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [15]:
full_table = confirmed_df_long.merge(right=deaths_df_long, how='left', on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])# Merging confirmed_df_long and deaths_df_long
full_table = full_table.merge(right=recovered_df_long, how='left', on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])# Merging full_table and recovered_df_long

In [16]:
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.15330,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0,0,0.0


In [17]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190120 entries, 0 to 190119
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Province/State  59073 non-null   object 
 1   Country/Region  190120 non-null  object 
 2   Lat             188762 non-null  float64
 3   Long            188762 non-null  float64
 4   Date            190120 non-null  object 
 5   Confirmed       190120 non-null  int64  
 6   Deaths          190120 non-null  int64  
 7   Recovered       175861 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 13.1+ MB


1. Convert date string to datetime
2. Replace NaN
3. Remove non-countries

In [18]:
full_table['Date'] = pd.to_datetime(full_table['Date'])

In [19]:
full_table.isna().sum()

Province/State    131047
Country/Region         0
Lat                 1358
Long                1358
Date                   0
Confirmed              0
Deaths                 0
Recovered          14259
dtype: int64

In [20]:
full_table['Recovered'] = full_table['Recovered'].fillna(0) #seems like a lot of courntries stopped reporting (recovered) data

In [21]:
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Province/State'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MSZaandam')
full_table = full_table[~ship_rows] #getting rid of ships

In [22]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188083 entries, 0 to 190119
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Province/State  57715 non-null   object        
 1   Country/Region  188083 non-null  object        
 2   Lat             186725 non-null  float64       
 3   Long            186725 non-null  float64       
 4   Date            188083 non-null  datetime64[ns]
 5   Confirmed       188083 non-null  int64         
 6   Deaths          188083 non-null  int64         
 7   Recovered       188083 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 12.9+ MB


In [23]:
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered'] # Active Case = confirmed - deaths - recovered

In [24]:
full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

<ipython-input-24-b4242cc93c67>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()


In [25]:
full_grouped.tail()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
132400,2021-11-30,Vietnam,1238082,25252,0.0,1212830.0
132401,2021-11-30,West Bank and Gaza,460053,4796,0.0,455257.0
132402,2021-11-30,Yemen,10004,1950,0.0,8054.0
132403,2021-11-30,Zambia,210169,3667,0.0,206502.0
132404,2021-11-30,Zimbabwe,134625,4707,0.0,129918.0


In [26]:
# new cases 
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan# renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])# filling na with 0
full_grouped = full_grouped.fillna(0)# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')# 
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

<ipython-input-26-feef5ac74c59>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']


In [27]:
full_grouped.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,2020-01-22,Afghanistan,0,0,0.0,0.0,0,0,0
1,2020-01-22,Albania,0,0,0.0,0.0,0,0,0
2,2020-01-22,Algeria,0,0,0.0,0.0,0,0,0
3,2020-01-22,Andorra,0,0,0.0,0.0,0,0,0
4,2020-01-22,Angola,0,0,0.0,0.0,0,0,0


In [28]:
full_grouped.tail(20)

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
132385,2021-11-30,Timor-Leste,19823,122,0.0,19701.0,1,0,0
132386,2021-11-30,Togo,26265,243,0.0,26022.0,15,0,0
132387,2021-11-30,Tonga,1,0,0.0,1.0,0,0,0
132388,2021-11-30,Trinidad and Tobago,71361,2158,0.0,69203.0,763,24,0
132389,2021-11-30,Tunisia,717572,25373,0.0,692199.0,263,8,0
132390,2021-11-30,Turkey,8797558,76842,0.0,8720716.0,25216,207,0
132391,2021-11-30,US,48554890,780561,0.0,47774329.0,116588,1445,0
132392,2021-11-30,Uganda,127550,3252,0.0,124298.0,27,0,0
132393,2021-11-30,Ukraine,3606622,91268,0.0,3515354.0,11212,595,0
132394,2021-11-30,United Arab Emirates,742041,2147,0.0,739894.0,65,1,0


In [29]:
full_grouped.to_csv('COVID-19-time-series-clean-complete.csv')